In [1]:
print("Hello World")

Hello World


In [2]:
pip install yfinance

Note: you may need to restart the kernel to use updated packages.Collecting yfinance
  Using cached yfinance-0.2.37-py2.py3-none-any.whl.metadata (11 kB)
  Using cached multitasking-0.0.11-py3-none-any.whl.metadata (5.5 kB)
  Using cached lxml-5.1.0-cp311-cp311-win_amd64.whl.metadata (3.6 kB)
  Using cached appdirs-1.4.4-py2.py3-none-any.whl.metadata (9.0 kB)
  Using cached frozendict-2.4.0-py3-none-any.whl
  Using cached peewee-3.17.1-py3-none-any.whl
  Using cached html5lib-1.1-py2.py3-none-any.whl.metadata (16 kB)
Using cached yfinance-0.2.37-py2.py3-none-any.whl (72 kB)
Using cached appdirs-1.4.4-py2.py3-none-any.whl (9.6 kB)
Using cached html5lib-1.1-py2.py3-none-any.whl (112 kB)
Using cached lxml-5.1.0-cp311-cp311-win_amd64.whl (3.9 MB)
Using cached multitasking-0.0.11-py3-none-any.whl (8.5 kB)




[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: C:\Users\Tanish Nagrani\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [3]:
import yfinance as yf
import pandas as pd
import csv
import datetime as dt
from datetime import datetime # To get the current date and time
from datetime import date, timedelta
import csv
import time
import os

Load OHLC data using yfinance

In [22]:
import yfinance as yf

def fetch_OHLC_data(symbol, start_date=None, end_date=None, output_file=None):
    # Fetch OHLC data using yfinance
    if start_date and end_date:
        data = yf.download(symbol, start=start_date, end=end_date)
    else:
        data = yf.download(symbol)

    if output_file:
        data.to_csv(output_file)

    return data

if __name__ == "__main__":
    # Define the symbol
    symbol = "IBM"  # Change symbol according to your input company
    start_date = "2023-01-01"
    end_date = "2024-03-26"
    output_file = "inputs/IBM_OHLC_data.csv"
    fetch_OHLC_data(symbol, start_date, end_date, output_file)
    print(f"OHLC data for {symbol} has been fetched and stored in {output_file}")


[*********************100%%**********************]  1 of 1 completed

OHLC data for IBM has been fetched and stored in inputs/IBM_OHLC_data.csv


Data Validation

In [32]:
from numpy import int64
import pandas as pd
import numpy as np
from scipy.stats import zscore

def validate_data_integrity(df):
    # Check for missing values
    missing_values = df.isnull().sum()
    if missing_values.any():
        print("Missing values found:")
        print(missing_values)

    # Check data types of essential columns
    expected_dtypes = {'Open': float, 'High': float, 'Low': float, 'Close': float, 'Volume': int64}
    for column, dtype in expected_dtypes.items():
        if df[column].dtype != dtype:
            print(f"Data type mismatch for column {column}. Expected: {dtype}, Actual: {df[column].dtype}")

def standardize_data_format(data_file):
    # Read the OHLC data from the CSV file
    df = pd.read_csv(data_file)

    # Ensure that the DataFrame has the required columns: Date, Open, High, Low, Close, Volume
    required_columns = ['Date', 'Open', 'High', 'Low', 'Close', 'Volume']
    missing_columns = set(required_columns) - set(df.columns)
    
    if missing_columns:
        print("Missing columns found. Adding missing columns with NaN values:")
        for column in missing_columns:
            df[column] = pd.NA

    # Reorder the columns to match the required structure
    df = df[required_columns]

    # Convert 'Date' column to datetime format
    df['Date'] = pd.to_datetime(df['Date'])

    # Identify and correct outliers using z-score method
    z_scores = np.abs(zscore(df[['Open', 'High', 'Low', 'Close']]))
    df = df[(z_scores < 3).all(axis=1)]

    return df

if __name__ == "__main__":
    # Load and standardize the OHLC data for each stock
    data_files = {
        "ALK": "inputs/ALK_OHLC_data.csv",
        "AMZN": "inputs/AMZN_OHLC_data.csv",
        "IBM": "inputs/IBM_OHLC_data.csv",
        "NFLX": "inputs/NFLX_OHLC_data.csv",
        "NVDA": "inputs/NVDA_OHLC_data.csv"
    }

    dataframes = {}
    for stock, data_file in data_files.items():
        dataframes[stock] = standardize_data_format(data_file)

    # Validate data integrity for each stock
    for stock, df in dataframes.items():
        print(f"Data integrity validation for {stock}:")
        validate_data_integrity(df)

    print("Data Validation Successful")

Data integrity validation for ALK:
Data integrity validation for AMZN:
Data integrity validation for IBM:
Data integrity validation for NFLX:
Data integrity validation for NVDA:
Data Validation Successful
